# CNN LSTM

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, LSTM

from sklearn.preprocessing import Normalizer

In [2]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [3]:
traindata = pd.read_csv(DATA+'kddcup.data.corrected.csv', header=None)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [4]:
traindata['Y'] = traindata.iloc[:, -1]
traindata.drop(columns=[len(traindata.columns)-2], inplace=True)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,Y
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [5]:
traindata = pd.get_dummies(traindata, columns=[1,2,3])
traindata

,0,4,5,6,7,8,9,10,11,12,...,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,0,215,45076,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,162,4528,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,236,1228,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,233,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,212,2288,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4898427,0,219,236,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4898428,0,218,3610,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4898429,0,219,1234,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
SIZE = traindata.shape[0]
testdata = traindata.iloc[int(SIZE-(SIZE*TEST)):, :]
traindata = traindata.iloc[:int(SIZE-(SIZE*TEST)), :]

In [7]:
traindata

,0,4,5,6,7,8,9,10,11,12,...,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,0,215,45076,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,162,4528,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,236,1228,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,233,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428896,0,36,117,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428897,0,48,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428898,65,94,31,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428899,0,48,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
testdata

,0,4,5,6,7,8,9,10,11,12,...,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
3428901,0,48,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428902,0,48,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428903,0,91,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428904,0,48,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3428905,11,48,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,212,2288,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4898427,0,219,236,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4898428,0,218,3610,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4898429,0,219,1234,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [10]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)

scaler = Normalizer().fit(T)
X_test = scaler.transform(T)

C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\hites\miniconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [11]:
y_train= pd.get_dummies(Y)
y_test= pd.get_dummies(C)

In [12]:
y_train.columns

Index(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.',
       'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.',
       'normal.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'rootkit.', 'satan.',
       'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'],
      dtype='object')

In [13]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [14]:
model = Sequential()
model.add(LSTM(16, input_shape=X_train.shape[1:]))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation="softmax"))

In [15]:
# define optimizer and objective, compile model
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [16]:
# train
model.fit(X_train, y_train, epochs=3, validation_split=0.4, shuffle=False, batch_size=128)

Epoch 1/3
16073/16073 [==============================] - 249s 15ms/step - loss: 0.2388 - accuracy: 0.9544 - val_loss: 0.2278 - val_accuracy: 0.9759
Epoch 2/3
16073/16073 [==============================] - 256s 16ms/step - loss: 0.1326 - accuracy: 0.9700 - val_loss: 0.2286 - val_accuracy: 0.9759
Epoch 3/3
16073/16073 [==============================] - 262s 16ms/step - loss: 0.1132 - accuracy: 0.9724 - val_loss: 0.2306 - val_accuracy: 0.9759


In [17]:
model.evaluate(X_test, y_test)

MemoryError: Unable to allocate 684. MiB for an array with shape (1469530, 122, 1) and data type float32

In [18]:
model.save(MODELS+"lstm_model.hdf5")